# Vanilla GAN MNIST

### Import Packages

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./MNIST_data/", one_hot= True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


### Set hyperparameters

In [5]:
total_epoch = 100
batch_size = 100
learning_rate = 0.001
n_hidden = 256
n_input = 28 * 28
n_noise = 128 # noise size for generator

### Set placeholder

GAN is also kind of unsupervised learning. You don't need to set label data Y. But you set two placeholder X and Z. X is for real data set and Z is for inputting noise to generate fake data. Both X and Z are fed to discriminator.

In [7]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

### Set variables for Generator

First variable(G_W1) and bias(G_b1) are variables for hidden layer and second variable(G_W2) and bias(G_b2) are for output. The shape of second variable is correspond to input image shape which enter to discriminator.

In [11]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev= 0.01)) # (n_noise, n_hidden)
G_b1 = tf.Variable(tf.zeros([n_hidden])) # (n_hidden,)
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01)) # (n_hidden, n_input)
G_b2 = tf.Variable(tf.zeros([n_input])) # (n_input,)

### Set variables for Discriminator

I just set the hidden layer of discriminator similar to generator. The output of discriminator is 0 or 1 in order to distinguish real or fake image. 

We must set the same variable in discriminator 1) which distinguish real image and 2) another discriminator which tell the generated image. Only if same variable is used when distinguising, Discriminator can figure out the real/fake image characteristics.

In [12]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev= 0.01)) # (n_input, n_hidden)
D_b1 = tf.Variable(tf.zeros([n_hidden])) # (n_hidden,)
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01)) # (n_hidden, 1)
D_b2 = tf.Variable(tf.zeros([1])) # (1,)

### Set generator & discriminator

In [22]:
def generator(noise_z):
    # input : noise_z (None, n_noise)
    # output : output (None, 784)    
    
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1) # (None, n_hidden)
    output = tf.nn.relu(tf.matmul(hidden, G_W2) + G_b2) # (None, 784)
    return output

def discriminator(inputs):
    # input : inputs (None, 784)
    # output : output (None, 1)
    
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1) # (None, n_hidden)
    output = tf.sigmoid(tf.matmul(hidden, D_W2) + D_b2) # (None, 1)
    
    return output

### Set noise generator

In [19]:
def get_noise(batch_size, n_noise):
    # output : random matrix (batch_size, n_noise)
    
    return np.random.normal(size = (batch_size, n_noise))

Generator creates fake image by using random noise z. Discriminator get two inputs, first, fake images by generator and second, real images. Discriminator judge what is real and fake.

In [23]:
G = generator(Z) # (None, 784)
D_gene = discriminator(G) # (None, 1)
D_real = discriminator(X) # (None, 1)

### Set loss

+ 1. loss which discriminator judges that fake image generated by generator is `fake.`
    - Example of counterfeit, this loss is relevent to **police.**
+ 2. loss which discriminator judges that fake image generated by generator is `real.`
    - Example of counterfeit, this loss is relevent to **counterfeiter**
    
For #1 loss of **police**, In order to train, D_real closes to '1'(discriminator judges that it's real) and D_gene closes to '0'(it's fake). SImply, `D_real` + `1- D_gene` makes loss.

In [24]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

For #2 loss of **counterfeiter**, In order to train, only D_gene should be closed to '1'. It means that discriminator judge that fake images as real or, Generator(counterfeiter) deceives Discriminator(police).

In [25]:
loss_G = tf.reduce_mean(tf.log(D_gene))

Object of GAN training is to maximize the loss_D & loss_G. But loss_D & loss_G are dependent. Accordingly, their increse or decrease relation is not proportional. Somtimes, they have inverse relation. Therefore they have `adversarial relation`.(So we call this network as adversarial network).

### Set optimizer

**※ Caution :** <br>
Only discriminator variables are to be used only if you get loss_D. On the other hand, only generator variables must be used only if you get loss_G. By seperating this two loss, Getting loss_G doesn't change the discriminator variables and vice versa.

In [29]:
D_var_list = [D_W1, D_W2, D_b1, D_b2]
G_var_list = [G_W1, G_W2, G_b1, G_b2]

According to the GAN paper, loss must become maximum. But in most of deep learning framework(Tensorflow, Keras...), They supports only minimize optimizer. Therefore we set the **minus** to the `loss_G` and `loss_D`

In [30]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

### Set train

In [ ]:
sess = tf.Session()